In [ ]:
!pip install -q streamlit torch transformers altair unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/

In [ ]:
%%writefile app.py
import streamlit as st
from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import get_chat_template
import re
# Load the model and tokenizer (shared across both pages)
@st.cache_resource
def load_model():
    max_seq_length = 2048
    dtype = None
    load_in_4bit = True

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/Llama-3.2-3B-Instruct",
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )

    tokenizer = get_chat_template(
        tokenizer, chat_template="llama-3.1"
    )
    FastLanguageModel.for_inference(model)  # Enable faster inference
    return model, tokenizer

model, tokenizer = load_model()

def elaborate_text(text_content):
    """
    Elaborates on the given text content using the Llama-3.2-3B-Instruct model.

    Args:
        text_content (str): The text content to elaborate on.

    Returns:
        str: The elaborated text.
    """

    # Create a conversation for elaboration
    conversation = [
        {"role": "system", "content": ""},
        {"role": "user", "content": f"Please elaborate on the following text:\n{text_content}"},
    ]

    # Apply chat template
    formatted_input = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=False)

    # Run inference
    with torch.no_grad():
        outputs = model.generate(
            input_ids=tokenizer(formatted_input, return_tensors="pt").input_ids.cuda(),
            max_new_tokens=1024,  # Adjust as needed
        )

    # Return elaborated output
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    assistant_output = re.search(r"Please elaborate on the following text:\n.*?\n(.*)", full_output, re.DOTALL)

    if assistant_output:
        return assistant_output.group(1).strip()
    else:
        return full_output.strip()


# Sidebar for page navigation
st.sidebar.title("Navigation")
page = st.sidebar.radio("Select a page:", ("Fake News Detector", "Chatbot"))

# Page 1: Fake News Detector
if page == "Fake News Detector":
    st.title("Fake News Detector")
    st.subheader("Enter a news statement to determine if it's real or fake.")

    # Input text from the user
    text_content = st.text_area(
        "News Statement:",
        placeholder="Enter a news statement (e.g., 'The Nile is the longest river in the world')",
    )

    if st.button("Analyze Statement"):
        if not text_content.strip():
            st.warning("Please enter a valid news statement!")
        else:
            # Construct the messages for the model
            messages = [
                {
                    "role": "system",
                    "content": """You are an expert in identifying fake news and disinformation. Please identify whether the piece of news is real or fake.
                    Please think step-by-step as you answer the question. However, please only respond with 'real' if the news is real or 'fake' if the news is fake.
                    Answer 'real' instead of 'true' and 'fake' instead of 'false'.""",
                },
                {"role": "human", "content": text_content},
            ]

            # Tokenize the input for prediction
            inputs = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
            ).to("cuda" if torch.cuda.is_available() else "cpu")

            # Generate the prediction
            output = model.generate(
                input_ids=inputs,
                max_new_tokens=128,
                use_cache=True,
                temperature=1.5,
                min_p=0.1,
            )

            # Decode the output
            decoded_output = tokenizer.decode(output[0], skip_special_tokens=True).strip()

            # Extract the prediction
            if "real" in decoded_output.lower():
                prediction = "real"
            elif "fake" in decoded_output.lower():
                prediction = "fake"
            else:
                prediction = "unknown"

            # Display the prediction
            st.success(f"The news statement is predicted to be: {prediction}")

            # Display the prediction
            # st.success(f"The news statement is predicted to be: {prediction}")

            # Generate an explanation or elaboration
            st.write("Generating an explanation...")
            explanation_messages = [
                {
                    "role": "system",
                    "content": f"Please elaborate on the following text:\n{text_content}",
                },
                {"role": "human", "content": text_content},
            ]



            # Display the explanation
            text_to_elaborate = text_content
            elaborated_text = elaborate_text(text_to_elaborate)
            st.info(f"Explanation: {elaborated_text}")

# Page 2: Chatbot
elif page == "Chatbot":
    st.title("Chatbot")
    st.subheader("Ask any question about your topic or something whihch you wanted to know :)")

    # Initialize session state to store conversation history
    if 'conversation_history' not in st.session_state:
        st.session_state['conversation_history'] = []

    # Input text from the user
    user_input = st.text_input("Ask your question here:")

    # If the user submits a question
    if st.button("Submit Question"):
        if user_input:
            # Add the user's question to the conversation history
            st.session_state['conversation_history'].append({"role": "human", "content": user_input})

            # Prepare the conversation history for the model
            conversation_input = [{"role": "system", "content": "You are a helpful assistant that answers questions based on the topic."}]
            conversation_input.extend(st.session_state['conversation_history'])

            # Tokenize and generate a response from the model
            inputs = tokenizer.apply_chat_template(
                conversation_input,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
            ).to("cuda" if torch.cuda.is_available() else "cpu")

            # Generate chatbot's response
            chatbot_output = model.generate(
                input_ids=inputs,
                max_new_tokens=128,
                use_cache=True,
                temperature=1.0,
                min_p=0.9,
            )

            # Decode the chatbot response
            chatbot_response = tokenizer.decode(chatbot_output[0], skip_special_tokens=True).strip()
            chatbot_response = chatbot_response.split("assistant")[-1].strip()

            # Display chatbot's response
            st.session_state['conversation_history'].append({"role": "assistant", "content": chatbot_response})
            st.write(f"Chatbot: {chatbot_response}")
        else:
            st.warning("Please enter a question to get an answer.")




Writing app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com


34.125.101.253


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.101.253:8501

⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏your url is: https://small-hounds-like.loca.lt
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2024-12-02 11:35:08.553559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 11:35:08.576737: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 11:35:08.583507: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plug